In [1]:
import sys
sys.path += [
    './lib/',
    './lib/field_mappers'
]

In [2]:
from pymongo import MongoClient
import spacy

from extract_biographies import extract_biographies
from generate_sample_collection import copy_sample
from extract_first_sentence import FirstSentenceExtractor
from vectorizer import Vectorizer
from pageview_aggregator import PageviewAggregator
from query import Searcher

In [3]:
SPACY_MODEL = 'en_vectors_web_lg'

In [4]:
db = MongoClient().who
# clean up test db
db.bios.remove({})
db.sample_bios.remove({})

/Users/lu/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/lu/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  after removing the cwd from sys.path.


{'n': 10, 'ok': 1.0}

In [5]:
extract_biographies(collection_name='bios', limit=100)

inserting batch 0


In [6]:
db.bios.count_documents({})

100

In [7]:
copy_sample('bios', 'sample_bios', 10)
db.sample_bios.count_documents({})

10

In [8]:
FirstSentenceExtractor().extract('sample_bios', new_field_name='first_sentence')

insertation count: 0


In [9]:
next(db.sample_bios.find({})).keys()

dict_keys(['_id', 'name', 'bio', 'first_sentence'])

In [10]:
next(db.sample_bios.find({}))['first_sentence']

'Irwin Allen Ginsberg  was an American poet, philosopher and writer.'

In [11]:
nlp = spacy.load(SPACY_MODEL)

In [12]:
Vectorizer(nlp).vectorize_text(
    collection_name='sample_bios',
    text_field_name='first_sentence',
    vector_field_name='fresh_new_vectorization'
)

insertation count: 0


In [13]:
pva = PageviewAggregator()
# do but once
# pva.aggregate('./data/monthly_view_totals/wump')
pva.load_totals()
pva.insert(collection_name='sample_bios')

pageview total insertation count: 0


In [14]:
next(db.sample_bios.find({}))['fresh_new_vectorization'][:9]

[-0.0013794583501294255,
 0.3548213839530945,
 0.13892532885074615,
 0.07493823766708374,
 0.19300685822963715,
 0.18327070772647858,
 0.01302451454102993,
 -0.10365469753742218,
 -0.15150092542171478]

In [15]:
next(db.sample_bios.find({})).keys()

dict_keys(['_id', 'name', 'bio', 'first_sentence', 'fresh_new_vectorization', 'views'])

In [16]:
Searcher(
    nlp,
    collection_name='sample_bios',
    vector_field_name='fresh_new_vectorization'
).query('gay poet', limit=10)

generating vector similarity maps...


['Allen Ginsberg',
 'Aleister Crowley',
 'Alfred Nobel',
 'Alexander III of Russia',
 'Alexander Agassiz',
 'Alfred Lawson',
 'Alfred Korzybski',
 'Amalaric',
 'Alexander Balas',
 'Alexander of Pherae']